## Calls for Service New Orleans Search
<p>
    <i>
        James Steele Howard, M.D. <br />
        2021/05/11 <br />
    </i>
</p>


<p>
    <b>Script Description:</b>
</p>
<p>
    This script uses data from all incidents reported to the New Orleans
    Police Department over the years of 2011 through 2020 to create statistical plots and maps by year. 
    Data sourced from <a href="https://datadriven.nola.gov/home/">DataDriven.NOLA.gov</a> "Calls for Service".
</p>
<p>
    <b>Dataset Notes:</b>
<p>
    To protect the privacy of victims, addresses were reported at the block level, and the calltypes Cruelty 
    to Juveniles, Juvenile Attachment, and Missing Juvenile were removed in accordance with the Louisiana 
    Public Records Act, L.R.S. 44:1.
</p>
<p>
    Calls for service regarding Aggravated Rape, Aggravated Rape - MA, Crime Against Nature, Mental Patient, 
    Oral Sexual Battery, Prostitution, Sexual Battery, Simple Rape, Simple Rape - Male V, and Soliciting for 
    Prost. are included in statistical plots. However, due to the removal of map coordinates (X,Y) for
    victim protective purposes, they are unable to included for mapping purposes or statistical calculations 
    involving mapped calls.
</p>

In [26]:
import os
import pandas as pd
import geopandas as gpd
import re

In [27]:
start_year = 2014
end_year = 2020
# Type of calls for service (used in plot titles and filenames). 
## No spaces, and use undercores to separate words.
call_types = 'domestic_violence' 
# Calls of interest from dataset
calls_of_interest = ['SIMPLE BATTERY DOMESTIC', 
                     'SIMPLE ASSAULT DOMESTIC', 
                     'AGGRAVATED ASSAULT DOMESTIC',
                     'AGGRAVATED BATTERY DOMESTIC',
                     'HOMICIDE DOMESTIC']

In [28]:
dir_main = os.getcwd()
dir_data = os.path.join(dir_main, 'data_source')
dir_cleaned = os.path.join(dir_main, 'data_cleaned')
dir_map_res = os.path.join(dir_main, 'map_resources')
dir_gj = os.path.join(dir_main, 'exported_geojsons')
dir_plots = os.path.join(dir_main, 'exported_plots')
dir_maps = os.path.join(dir_main, 'exported_maps')

#### Set the start and end years for CSVs to be read into dataframes

In [29]:
year = start_year
df_list = []
while year <= end_year:
    df_calls = pd.read_csv(os.path.join(dir_data, f'Calls_for_Service_{year}.csv'), low_memory=False)
    df_list.append(df_calls)
    year += 1 

#### Add year column for each dataframe, populate year column, and drop unneeded columns

In [30]:
dfs_cleaned = []
year_temp = start_year
for df in df_list:
    df['Year'] = year_temp
    df = df[['Year', 'TypeText', 'MapX', 'MapY']]
    dfs_cleaned.append(df)
    year_temp +=1

<b><font color="red">Markdown in RED font are not needed for script and are only included as a representation of the process in Python for filtering dataset for certain types of calls.</font></b>

### <font color="red">View unique values for the type of police call</font>

In [31]:
unique = dfs_cleaned[0]['TypeText'].unique().tolist()
unique

['AGGRAVATED BATTERY BY SHOOTING',
 'COMPLAINT OTHER',
 'MEDICAL',
 'DISCHARGING FIREARM',
 'FIREWORKS',
 'DISTURBANCE (OTHER)',
 'BURGLAR ALARM, SILENT',
 'FIRE',
 'SUSPICIOUS PERSON',
 'HIT & RUN',
 'AUTO ACCIDENT',
 'PICKPOCKET',
 'MENTAL PATIENT',
 'UNDERAGE DRINKING VIOLATION',
 'LOST PROPERTY',
 'TRAFFIC INCIDENT',
 'MISSING ADULT',
 'DEATH',
 'OBSCENITY, EXPOSING',
 'AGGRAVATED CRIMINAL DAMAGE',
 'THEFT BY FRAUD',
 'DOMESTIC DISTURBANCE',
 'CURFEW VIOLATION',
 'WRECKLESS DRIVING',
 'SIMPLE BATTERY DOMESTIC',
 'ARMED ROBBERY WITH GUN',
 'PROWLER',
 'NOISE COMPLAINT',
 'UNAUTHORIZED USE OF VEHICLE',
 'AGGRAVATED ASSAULT',
 'SIMPLE CRIMINAL DAMAGE',
 'SIMPLE BATTERY',
 'FIGHT',
 'AUTO THEFT',
 'THEFT',
 'HIT & RUN WITH INJURIES',
 'FUGITIVE ATTACHMENT',
 'RETURN FOR ADDITIONAL INFO',
 'DRIVING WHILE UNDER INFLUENCE',
 'MISSING JUVENILE',
 'AGGRAVATED BATTERY',
 'AUTO ACCIDENT WITH INJURY',
 'SILENT 911 CALL',
 'CARJACKING',
 'MUNICIPAL ATTACHMENT',
 'DRUG VIOLATIONS',
 'DAILY WALKI

#### <font color="red">Compile a regular expression to only search for expressions with 'Domestic' in the type of police call</font>

In [32]:
r = re.compile('.*DOMESTIC')
new_unique = list(filter(r.match, unique))
new_unique

['DOMESTIC DISTURBANCE',
 'SIMPLE BATTERY DOMESTIC',
 'CRIMINAL DAMAGE DOMESTIC',
 'EXTORTION (THREATS) DOMESTIC',
 'AGGRAVATED BATTERY DOMESTIC',
 'AGGRAVATED ASSAULT DOMESTIC',
 'SIMPLE ASSAULT DOMESTIC',
 'CRIMINAL MISCHIEF DOMESTIC',
 'SIMPLE BURGLARY DOMESTIC',
 'HOMICIDE DOMESTIC',
 'SIMPLE ARSON DOMESTIC']

#### Create a list of call types of interest, restrict dataset to only call types of interest, and save csv

In [33]:
def restrict_calls(df_list, call_type):
    '''
    Filters calls to call_type list and concatenates dataframe list together.
    '''
    df_list_years_temp = []
    for df in df_list:
        df_list_calls_temp = []
        for call in call_type:
            df_temp = df[df['TypeText'] == call]
            #df_temp = df_temp.copy(deep=True) # Was getting a warning about using a copied dataframe. This fixes it.
            df_list_calls_temp.append(df_temp)
        df_year = pd.concat(df_list_calls_temp)
        df_list_years_temp.append(df_year)
    df_final_concat = pd.concat(df_list_years_temp)
    return df_final_concat

In [34]:
df_fully_cleaned = restrict_calls(dfs_cleaned, calls_of_interest)

In [35]:
output_filename = f'{start_year}-{end_year}_calls_for_{call_types}'
df_fully_cleaned.to_csv(os.path.join(dir_cleaned, f'{output_filename}.csv'), index=False)

#### Get total number of domestic abuse related calls by year

In [36]:
df = df_fully_cleaned.copy(deep=True)

In [37]:
df_grouped_year = df.groupby('Year').count()
df_grouped_year = df_grouped_year.reset_index()

#### Plot the calls by year

In [38]:
import plotly.graph_objects as go

In [52]:
plot = go.Figure(data=[go.Bar(x=df_grouped_year['Year'],
                              y=df_grouped_year['TypeText'],
                              marker_color='#c4382d')])
plot.update_layout(font_color='black',
                   title_font_family='Arial, sans-serif',
                   showlegend=False,
                   plot_bgcolor='#ededed',
                   margin=dict(l=0, r=0, b=0, t=0),
                   hovermode='x',
                   hoverlabel=dict(bgcolor='#f3e20c',
                                   font_size=15))

y_title = '<b>Calls for Service: ' + str.title(call_types).replace('_', ' ') + '</b>'
plot.update_yaxes(title_text=y_title, showspikes=False, spikecolor='#6f6f6f')
plot.update_xaxes(title_text='<b>Year</b>', showspikes=False, spikecolor='#6f6f6f')

plot.update_traces(marker=dict(line=dict(width=2, color='black')))

plot.write_html(os.path.join(dir_plots, f'{start_year}_{end_year}_{call_types}.html'))

#### Set projection to WGS84 and export GeoJSON

In [15]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.MapX, df.MapY))

In [16]:
gdf = gdf.set_crs('EPSG:3452')
gdf = gdf.to_crs('EPSG:4326')

In [17]:
gdf.to_file(os.path.join(dir_cleaned, f'{output_filename}.geojson'), driver='GeoJSON')

#### Split calls by year and add dataframes to list

In [18]:
year_list = gdf['Year'].unique().tolist()
year_list.sort() 

In [19]:
gdf_list = []
for year in year_list:
    vars()[f'gdf_{year}'] = gdf[gdf['Year'] == year]
    gdf_list.append(vars()[f'gdf_{year}'])

#### Spatially join calls with New Orleans neighborhoods

In [20]:
hoods = gpd.read_file(os.path.join(dir_map_res, 'neighborhoods.geojson'))
#hoods = gpd.read_file(os.path.join(dir_map_res, 'Neighborhoods', 'Neighborhood_Statistical_Areas', 'Neighborhood_Statistical_Areas.shp'))
hoods = hoods[['GNOCDC_LAB', 'geometry']]
hoods.rename(columns={'GNOCDC_LAB': 'neighborhood'}, inplace=True)
#hoods = gdf.to_crs('EPSG:4326')

In [21]:
gdf_list_temp = []
for gdf in gdf_list:
    gdf_temp = gpd.sjoin(hoods, gdf, how='left', op='contains')
    gdf_temp = gdf_temp[['Year', 'neighborhood', 'TypeText', 'geometry']]
    gdf_list_temp.append(gdf_temp)
gdf_list = gdf_list_temp

#### Get count of number of calls per neighborhoods 

In [22]:
gdf_agg_list = []
for gdf in gdf_list:
    gdf_temp = gdf.dissolve(by='neighborhood', aggfunc='count')
    gdf_temp = gdf_temp[['Year', 'geometry']]
    gdf_temp = gdf_temp.rename(columns={'Year': 'call_count'})
    gdf_agg_list.append(gdf_temp)

#### Save the cleaned geojson files

In [23]:
year_temp = start_year
for gdf in gdf_agg_list:
    gdf.to_file(os.path.join(dir_gj, f'{year_temp}_{call_types}_by_neighborhood.geojson'), driver='GeoJSON')
    year_temp += 1

#### Create Folium maps by year

In [24]:
import folium
import folium.plugins as plugins

In [25]:
year_temp = start_year
for gdf in gdf_agg_list:
    
    m = folium.Map(location=[30.029, -89.8688], zoom_start=11, max_bounds=True)
    folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
    
    layer_title = str.title(call_types).replace('_', ' ') + f' {year_temp}'
    #legend_title = legend_title.replace('_', ' ')
    gdf = gdf.reset_index() #######
    choropleth = folium.Choropleth(
                    geo_data=gdf,
                    name=layer_title,
                    data=gdf,
                    columns=['neighborhood', 'call_count'],
                    key_on='feature.properties.neighborhood',
                    bins=9,
                    fill_color='Reds',
                    nan_fill_color='black',
                    fill_opacity=0.8,
                    nan_fill_opacity=0,
                    line_color='black',
                    dash_array=10,
                    line_weight=1,
                    line_opacity=0.8,
                    legend_name=f'Number of Calls for Service: {layer_title}',
                    highlight=True,
                    smooth_factor=1
                    ).add_to(m)
    folium.LayerControl().add_to(m)
    folium.plugins.Fullscreen(position='topright').add_to(m)
    # Create popup layer
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#6f6f6f', 
                                'fillOpacity': 0, 
                                'weight': 0.1
                               }
    highlight_function = lambda x: {'fillColor': '#f3e20c',
                                    'color':'black',
                                    'fillOpacity': 0.8,
                                    'weight': 3
                                   }

    popup = folium.features.GeoJson(
        gdf,
        style_function=style_function,
        control=False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(fields=['neighborhood',
                                                       'call_count'
                                                      ],
                                               aliases=['Neighborhood:',
                                                        '<font color="#f3e20c">Calls:</font>'
                                                       ],
                                               style=("background-color: grey; color: #ffffff; font-family: arial; font-size: 12px; padding: 10px;")
                                              )
        )
    m.add_child(popup)
    m.keep_in_front(popup)
    m.save(os.path.join(dir_maps, f'{year_temp}_{call_types}_choropleth.html'))
    year_temp += 1